In [ ]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


In [ ]:
syscalls = [
    "open", "read", "write", "close", "exec", "connect", "accept", "fork",
    "kill", "chmod", "chown", "unlink", "reboot", "mount", "ptrace", "bind"
]

syscall_to_int = {sc: i+1 for i, sc in enumerate(syscalls)}  # 0 is reserved for padding
